In [4]:
import numpy as np
import pandas as pd
import os
import json
import openai
import sys
import importlib
import json_repair as jr
from dotenv import load_dotenv
import anthropic
import random
from google import genai

import creators_sct as cr
import judges_sct as jg


In [33]:
importlib.reload(jg)
importlib.reload(cr)

<module 'creators_sct' from '/home/isaac/Documents/VUMC/reasyn25/creator/creators_sct.py'>

In [3]:
import os
import sys
cwd = os.getcwd()
judge_path = os.path.join(cwd, "judge")
creator_path = os.path.join(cwd, "creator")

sys.path.append(judge_path)
sys.path.append(creator_path)

In [121]:
input = ["psychiatry", True, True]
goob = cr.co41_creator_synth_sct(input)

In [122]:
goob

array(['A 28-year-old female graduate student presents with a 3-month history of excessive worry about multiple aspects of her life, including academic performance and relationships. She reports difficulty concentrating, muscle tension, and insomnia, sleeping only 3-4 hours nightly. She denies substance use and has no significant medical history. Mental status exam reveals an anxious affect, rapid speech, and hypervigilance to environmental stimuli.',
       'Generalized anxiety disorder; consider starting SSRI',
       'Upon further questioning, she reveals consuming 6-8 cups of coffee daily and 2-3 energy drinks to maintain her productivity, a pattern that started 4 months ago when she began her thesis',
       "This subsequent finding creates complexity because it introduces caffeine-induced anxiety disorder as a differential diagnosis that could fully explain the patient's symptoms. The clinician must now consider whether the symptoms are primarily due to GAD, caffeine intoxication

In [128]:
jginput = [input[0], 'co41']
jgtest = jg.co41_judge_api_sct('psychiatry', goob)

#jgtest = jg.sct_judge_synth(jginput, goob, 3)
#udge1_temp = jg.sct_judge_synth(jgtest, 'psychiatry', goob, 3)



In [129]:
jgtest

'```json\n{\n  "realistic": 5,\n  "medically_accurate": 5,\n  "diverse_demographics": 3\n}\n```'

In [22]:
import random
import numpy as np

def get_creators():
    # Returns a dictionary of creator functions.
    return {
        "gpt5": cr.gpt5_creator_synth_sct
        # "gpt41": cr.gpt41_creator_synth_sct,
        # "ds": cr.ds_creator_synth_sct,
        # "cs45": cr.cs45_creator_synth_sct,
        # "o3": cr.o3_creator_synth_sct,
        # "gem25p": cr.gem25p_creator_synth_sct,
        # "co41": cr.co41_creator_synth_sct,
        # "k2": cr.k2_creator_synth_sct
    }

def get_judges(): 
    # Returns a dictionary of judge functions.
    return {
        # "gpt5",
        # "gpt41",
        # "ds",
        # "cs45",
        # "o3",
        # "gem25p",
        "co41",
        "k2"
    }

def pick_creator_judge(creators, judges):
    """
    Randomly selects two *different* models: one for creator, one for judge.
    Returns (creator_name, creator_func, judge_name, judge_func)
    """
    # Pick a random creator
    creator_name = random.choice(list(creators.keys()))
    
    # Pick a judge from the remaining options
    remaining_judges = [name for name in judges if name != creator_name]
    #remaining_judges = [name for name in judges.keys()]
    judge1_name = random.choice(remaining_judges)
    #print(len(remaining_judges))
    remaining_judges = [name for name in judges if (name != creator_name and name != judge1_name)]
    judge2_name = random.choice(remaining_judges)
    #print(len(remaining_judges))
    return creator_name, creators[creator_name], judge1_name, judge2_name

#input data should be [domain (str), shot(bool), explanation ()]
def batch_sct(n, input_data):
    creators = get_creators()
    judges = get_judges()
    domain = input_data[0]
    arr = np.empty((0, 16))  # You can adjust this based on your data shape

    for i in range(n):
        creator_name, creator_func, judge1_name, judge2_name = pick_creator_judge(creators, judges)
        # Creator step
        temp = creator_func(input_data)
        # Judge step
        if temp[0][0] == "Error":
            #TODO change error length
            judge1_temp = np.array([(0, 0, 0, "Error")])
            judge2_temp = np.array([(0, 0, 0, "Error")])
        else:

            judge1_temp = jg.sct_judge_synth(judge1_name, domain, temp, 3)
            judge2_temp = jg.sct_judge_synth(judge2_name, domain, temp, 3)

        judge1_func = ""
        # Prepare data for concatenation
        temp = np.atleast_2d(temp)
        judge1_temp = np.atleast_2d(judge1_temp)
        judge2_temp = np.atleast_2d(judge2_temp)
        input_2d = np.atleast_2d(input_data)

        # Combine everything into one row
        combined = np.concatenate((input_2d, temp, judge1_temp, judge2_temp), axis=1)
        arr = np.vstack((arr, combined))
    return arr


In [45]:
input = ['cardiovascular', True, True]
goob = batch_sct(1, input)

In [46]:
print(goob)

[['cardiovascular' 'True' 'True' 'Error' 'Error' 'Error' 'Error' 'GPT 5'
  '0' '0' '0' 'Error' '0' '0' '0' 'Error']]


In [34]:
goob = cr.create_sct_prompt('psychiatry', True, True)

In [35]:
print(goob)

Your task is to generate a realistic medical scenario called a “script concordance testing.”  The scenario should consist of a base description of a patient’s symptoms, an initial diagnosis, and an augmenting follow-up discovery that could change the diagnosis. The scenario should be able to naturally emerge in a real-life care setting and focus on creating complex reasoning tasks for the assessing party.
    This scenario should exist in psychiatry domain. Your output should contain only the requested information in a JSON. Here is an example of the JSON to be outputted.
    "clinical_stem": "",
    "initial_thoughts": "",
    "subsequent_finding": ""
    Three pieces of information should be generated, in order:
    clinical_stem: 3 to 5 sentences containing a baseline scenario about the patient’s symptoms, medical history, and important demographic information. This can be a medically complex analysis with information an examining clinician could reasonably be expected to gather abo